In [1]:
import pandas as pd

data = pd.read_csv('connectionlossdata.csv',low_memory=False)
data.head()

,Unnamed: 0,TOPLAMSAAT,SORUNSAYISINET,PortErrorCount,LostCarrierCount,GecenAyPortErrorSayisi,GecmisFark,Kopma,Atlak,POSSEHIR,POSSEMT,SANTRALADI,GeneralDate,anonimid
0,0,22,1,0,0,0,0,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-1-54,2020-12-01,14216756
1,1,22,1,0,0,0,0,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-İSTASYON-54,2020-12-01,14216756
2,2,46,3,1,0,0,1,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-1-54,2020-12-02,14216756
3,3,46,3,1,0,0,1,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-İSTASYON-54,2020-12-02,14216756
4,4,50,3,1,0,0,1,NaN,NaN,Edi̇rne,ŞÜKRÜPAŞA,EDİRNE-1-54,2020-12-03,14216756


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4835379 entries, 0 to 4835378
Data columns (total 14 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   Unnamed: 0              int64 
 1   TOPLAMSAAT              int64 
 2   SORUNSAYISINET          int64 
 3   PortErrorCount          int64 
 4   LostCarrierCount        int64 
 5   GecenAyPortErrorSayisi  int64 
 6   GecmisFark              int64 
 7   Kopma                   object
 8   Atlak                   object
 9   POSSEHIR                object
 10  POSSEMT                 object
 11  SANTRALADI              object
 12  GeneralDate             object
 13  anonimid                int64 
dtypes: int64(8), object(6)
memory usage: 516.5+ MB


In [4]:
# decide which categorical variables you want to use in model
for col_name in data.columns:
    if (data[col_name].dtypes == 'object'):
        unique_cat = len(data[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))

Feature 'Kopma' has 3 unique categories
Feature 'Atlak' has 2 unique categories
Feature 'POSSEHIR' has 212 unique categories
Feature 'POSSEMT' has 7550 unique categories
Feature 'SANTRALADI' has 5451 unique categories
Feature 'GeneralDate' has 13 unique categories


In [5]:
# we can remove the unnecessary features.
data = data.drop('Unnamed: 0',1)
data = data.drop('anonimid',1)


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4835379 entries, 0 to 4835378
Data columns (total 12 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   TOPLAMSAAT              int64 
 1   SORUNSAYISINET          int64 
 2   PortErrorCount          int64 
 3   LostCarrierCount        int64 
 4   GecenAyPortErrorSayisi  int64 
 5   GecmisFark              int64 
 6   Kopma                   object
 7   Atlak                   object
 8   POSSEHIR                object
 9   POSSEMT                 object
 10  SANTRALADI              object
 11  GeneralDate             object
dtypes: int64(6), object(6)
memory usage: 442.7+ MB


In [70]:
# datadaki illerin dağılımınada bakalım, istanbul, ankara ve izmir domine ediyor.
data['POSSEHIR'].value_counts().sort_values(ascending=False).head(10)

İstanbul    2012813
Ankara       347797
İzmir        237695
Bursa        203369
Antalya      139541
Kocaeli      126618
Konya         97457
Sakarya       85617
Muğla         81213
Adana         70171
Name: POSSEHIR, dtype: int64

In [69]:
print(data['POSSEHIR'].unique())

['Edi̇rne' 'İstanbul' 'İzmir' 'Antalya' 'Samsun' 'Aydin' 'Ankara' 'Manisa'
 'Trabzon' 'Sivas' 'Si̇vas' 'Hatay' 'Aydın' 'Adana' 'The netherlands'
 'Gaziantep' 'Kocaeli' 'Bursa' 'Konya' 'Mersi̇n' 'Bolu' 'Kars'
 'Kırklareli' 'Denizli' 'Muğla' 'Tekirdağ' 'Kayseri' 'Uşak' 'Nevşehir'
 'Kayseri̇' 'Balıkesir' 'Nevşehi̇r' 'Burdur' 'Erzurum' 'Eskişehir' 'Afyon'
 'Zonguldak' 'Kastamonu' 'Malatya' 'Kahramanmaraş' 'Mardi̇n' 'Edirne'
 'Isparta' 'Çanakkale' 'Sinop' 'Diyarbakır' 'Kocaeli̇' 'Şırnak' 'Düzce'
 'Yozgat' 'Yalova' 'Ağri' 'Kütahya' 'Düzce ' 'Ordu' 'Çankiri ' 'Rize'
 'Sakarya' 'Giresun' 'Çankırı' 'Ri̇ze' 'Bartın' 'Trabzon ' 'Erzurum '
 'Eski̇şehi̇r' 'Erzincan' 'Bi̇leci̇k' 'Elazığ' 'Şanlıurfa' 'Isparta '
 'Osmaniye' 'Karaman' 'Karabük' 'Kirklareli̇' 'Eski̇şehi̇r ' 'Bilecik'
 'Adıyaman' 'Çorum' 'Tokat' 'Bayburt' 'Amasya' 'Deni̇zli̇' 'Adiyaman'
 'Tunceli' 'Si̇nop ' 'Gümüşhane' 'Kırşehir' 'Niğde' 'Gazi̇antep' 'Artvin'
 'Mardin' 'Aksaray ' 'Batman' 'Muş' 'Gi̇resun' 'Ordu ' 'Aksaray' 'Tokat '
 'Erz

In [67]:
# şehir isimlerini düzeltme
data['POSSEHIR'] = data['POSSEHIR'].replace(['İstanbul (anadolu)','İstanbul (avrupa)','İstanbul ','Istanbul'],'İstanbul')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Ankara '],'Ankara')
data['POSSEHIR'] = data['POSSEHIR'].replace(['İzmi̇r '],'İzmir')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Bursa '],'Bursa')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Si̇vas '],'Sivas')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Antalya '],'Antalya')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Kocaeli̇ '],'Kocaeli')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Sakarya '],'Sakarya')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Konya '],'Konya')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Muğla '],'Muğla')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Adana '],'Adana')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Balıkesir ','Balikesi̇r','Balikesi̇r '],'Balıkesir')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Teki̇rdağ','Teki̇rdağ '],'Tekirdağ')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Mani̇sa','Mani̇sa '],'Manisa')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Çanakkale '],'Çanakkale')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Hatay '],'Hatay')
data['POSSEHIR'] = data['POSSEHIR'].replace(['Mersin','Mersi̇n '],'Mersi̇n')

In [68]:
# datadaki illerin dağılımınada bakalım, istanbul, ankara ve izmir domine ediyor.
data['POSSEHIR'].value_counts().sort_values(ascending=False).head(30)

İstanbul         2012813
Ankara            347797
İzmir             237695
Bursa             203369
Antalya           139541
Kocaeli           126618
Konya              97457
Sakarya            85617
Muğla              81213
Adana              70171
Balıkesir          67291
Tekirdağ           59853
Hatay              57388
Manisa             54188
Çanakkale          52277
Mersi̇n            49702
Sivas              29002
Samsun             27322
Denizli            25400
Eskişehir          23826
Kahramanmaraş      21177
Kayseri            20964
Kırklareli         20129
Gaziantep          19587
Aydın              19496
Isparta            19080
Kütahya            18061
Eski̇şehi̇r        16633
Deni̇zli̇          16491
Aydin              16103
Name: POSSEHIR, dtype: int64

In [9]:
data['Atlak'].value_counts().sort_values(ascending=False).head(10)

Atlak Bağlantı Sorunu    2
Name: Atlak, dtype: int64

In [ ]:
# Also 'POSSEMT', 'SANTRALADI' features has too many unique categories. We can remove them.
data = data.drop('POSSEMT',1)
data = data.drop('SANTRALADI',1)

In [30]:
# we can change the 'İstanbul (avrupa)' and 'İstanbul (anadolu)' as 'İstanbul'


data.replace('İstanbul (avrupa)','Istanbul')
data.replace('İstanbul ','Istanbul')
data.replace('İstanbul','Istanbul')

,TOPLAMSAAT,SORUNSAYISINET,PortErrorCount,LostCarrierCount,GecenAyPortErrorSayisi,GecmisFark,Kopma,Atlak,POSSEHIR,GeneralDate
0,22,1,0,0,0,0,NaN,NaN,Edi̇rne,2020-12-01
1,22,1,0,0,0,0,NaN,NaN,Edi̇rne,2020-12-01
2,46,3,1,0,0,1,NaN,NaN,Edi̇rne,2020-12-02
3,46,3,1,0,0,1,NaN,NaN,Edi̇rne,2020-12-02
4,50,3,1,0,0,1,NaN,NaN,Edi̇rne,2020-12-03
...,...,...,...,...,...,...,...,...,...,...
4835374,12,2,1,0,0,1,NaN,NaN,Bursa,2020-12-13
4835375,0,2,0,0,0,0,NaN,NaN,İstanbul (anadolu),2020-12-13
4835376,4,2,0,0,0,0,NaN,NaN,Kocaeli̇,2020-12-13
4835377,11,1,0,0,0,0,NaN,NaN,Ankara,2020-12-13


In [31]:
# datadaki illerin dağılımınada bakalım, istanbul, ankara ve izmir domine ediyor.
data['POSSEHIR'].value_counts().sort_values(ascending=False).head(10)

İstanbul              813308
İstanbul (avrupa)     722512
İstanbul (anadolu)    364610
Ankara                219765
Ankara                128032
İzmir                 125185
Bursa                 122742
İstanbul              112279
İzmi̇r                 94605
Antalya                84963
Name: POSSEHIR, dtype: int64

In [15]:
unique_cat = len(data['Kopma'].unique())
print("unique category '{cat}'".format(cat=unique_cat))
print(data['Kopma'].unique())

unique category '3'
[nan 'Sık Sık Kopuyor' 'İnternete Erişemiyor']
